In [9]:
import torchio as tio
from torchio.data.io import sitk_to_nib
import SimpleITK as sitk
import napari
import numpy as np
import nibabel as nib

from utils.image import read_im_gt, read_reorient_nifti
from utils.analysis import compute_dice

In [2]:
img_path = '/home/t722s/Desktop/Datasets/Dataset350_AbdomenAtlasJHU_2img/imagesTr/BDMAP_00000001_0000.nii.gz'
gt_path = '/home/t722s/Desktop/Datasets/Dataset350_AbdomenAtlasJHU_2img/labelsTr/BDMAP_00000001.nii.gz'
class_label = 3

gt_unprocessed = nib.load(gt_path).get_fdata()
gt_unprocessed = np.where(gt_unprocessed == class_label, 1, 0)

img, gt = read_im_gt(img_path, gt_path, class_label, RAS=True)

gt_2, _ = read_reorient_nifti(gt_path, np.float32, RAS = True)
gt_2 = np.where(gt_2 == 3, 1, 0)

np.array_equal(gt, gt_2)


NameError: name 'nib' is not defined

In [12]:
# Attempt at merged dice
seg = '/home/t722s/Desktop/ExperimentResults_lesions/segvol_melanoma_HD_sub/bbox3d/segmentationsTr/merged/0000380855_Baseline.nii.gz'
seg = nib.load(seg).get_fdata()

gt = '/home/t722s/Desktop/Datasets/melanoma_HD_sub/labelsTr/0000380855_Baseline.nii.gz'
gt = nib.load(gt).get_fdata()
compute_dice(seg, gt)

0.3946806518074546

In [11]:
# Attempt at instance dice
gt = '/home/t722s/Desktop/Datasets/melanoma_HD_sub/labelsTr/0000380855_Baseline.nii.gz'
gt = nib.load(gt).get_fdata()
gt = np.where(gt == 1, 1, 0)
seg = '/home/t722s/Desktop/ExperimentResults_lesions/segvol_melanoma_HD_sub/bbox3d/segmentationsTr/0000380855_Baseline/instance_1.0_seg.nii.gz'
seg = nib.load(seg).get_fdata()

compute_dice(seg, gt)

0.7686022760431864

In [1]:
import numpy as np
np.mean(np.array([1,3,None]))

TypeError: unsupported operand type(s) for +: 'int' and 'NoneType'

In [15]:
t = 'aaaaaba'
t = t.removesuffix('a')
t

'aaaaab'